# Final Report
By: Jenny Zhang, Marcus Kang, Joshua Chang, Daniel Lee

# Introduction

# Data Description

# Pre-registered Analyses

## Hypothesis 1: 
Yards per Reception and Yards after the Catch are most indicative of good post-season rank for a wide receiver.

## Hypothesis 2:
The number of points scored by wide receivers has increased over time.

# Data Analysis

## Data Cleaning

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
wr_df = pd.read_csv("wr_df.csv")
print(wr_df)

               Player  Year   Y/R  YBC/R  AIR/R  YAC/R  YACON/R  10+ YDS  \
0      DeVante Parker  2018  12.9    9.0    9.0    3.9      1.5     11.0   
1       Tyler Lockett  2017  12.3    8.2    8.2    4.2      0.4     20.0   
2       Davante Adams  2016  13.3    8.1    8.1    5.1      2.2     37.0   
3      Michael Thomas  2016  12.4    7.1    7.1    5.3      1.8     48.0   
4        Doug Baldwin  2015  13.7    8.1    8.1    5.6      2.2     40.0   
5    Michael Crabtree  2015  10.8    7.7    7.7    3.1      1.4     35.0   
6       Davante Adams  2015   9.7    6.9    6.9    2.7      1.1     16.0   
7   Odell Beckham Jr.  2014  14.3    9.0    9.0    5.3      0.0     45.0   
8        Doug Baldwin  2014  12.5    7.4    7.4    5.1      0.0     34.0   
9    Michael Crabtree  2014  10.3    6.5    6.5    3.8      0.0     25.0   
10     Julian Edelman  2013  10.1    5.3    5.3    4.7      0.0     39.0   

    20+ YDS  30+ YDS  40+ YDS  50+ YDS  Preseason_rank    ADP  \
0       3.0      3.0  

## Summary Statistics

### Post Season Rank

### Scoring Points Over Time

# Models

## Model (1)

## Model (2)

# Evaluation of Significance

## Model (1)

## Model (2)

# Interpretations and Conclusions

## Model (1)
## Model (2)

## Conclusions

# Limitations

## General

## Model (1)

## Model (2)